# **MEEN 423: Machine Learning for Mechanical Engineers**
# Homework 5: Decision Trees and Ensemble Methods

## Tasks

### 1. Train and fine-tune a Decision Tree for the moons dataset by following these steps:

#### A. Use make_moons (n_samples=10000, noise=0.4) to generate a moons dataset.

In [60]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Generate moons dataset
X, y = make_moons(n_samples=10000, noise=0.4, random_state=23)

#### B. Use train_test_split() to split the dataset into a training set and a test set.

In [61]:
# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

#### C. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. (Hint: try various values for max_leaf_nodes)

In [62]:
# Define hyperparameters to tune
param_grid = {'max_leaf_nodes': [None, 2, 4, 6, 8, 10, 15, 20, 25, 30, 40, 50]}

# Grid search with cross-validation
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=23), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get best hyperparameters
best_params = grid_search.best_params_

#### D. Train it on the full training set using these hyperparameters, and measure your model's performance on the test set. You should get roughly 85% to 87% accuracy.

In [63]:
# Train Decision Tree on full training set using best hyperparameters
best_dt = DecisionTreeClassifier(**best_params, random_state=23)
best_dt.fit(X_train, y_train)

# Measure model's performance on the test set
y_pred = best_dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Best hyperparameters:", best_params)
print("Test set accuracy: {:.2f}%".format(accuracy * 100))

Best hyperparameters: {'max_leaf_nodes': 15}
Test set accuracy: 85.85%


### 2. Grow a forest by following these steps:

#### A. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. (Hint: you can use Scikit-Learn’s ShuffleSplit class for this.)

In [64]:
from sklearn.model_selection import ShuffleSplit
from scipy.stats import mode

# Generate subsets of the training set
n_trees = 1000
subset_size = 100
shuffle_split = ShuffleSplit(n_splits=n_trees, test_size=subset_size, random_state=23)

#### B. Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree.

In [65]:
# Train one Decision Tree on each subset
trained_trees = []
for train_index, _ in shuffle_split.split(X_train):
    subset_X_train, subset_y_train = X_train[train_index], y_train[train_index]
    tree = DecisionTreeClassifier(**best_params, random_state=23)
    tree.fit(subset_X_train, subset_y_train)
    trained_trees.append(tree)

#### C. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set.

In [66]:
# Evaluate the 1000 Decision Trees on the test set
predictions = []
for tree in trained_trees:
    y_pred = tree.predict(X_test)
    predictions.append(y_pred)

# Generate majority-vote predictions
ensemble_predictions = mode(predictions, axis=0)[0].flatten()

#### D. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [67]:
# Evaluate the ensemble predictions on the test set
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print("Ensemble Test set accuracy: {:.2f}%".format(ensemble_accuracy * 100))

Ensemble Test set accuracy: 86.10%
